In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
from tqdm import tqdm
from glob import glob

Loading Ehist Files

In [4]:
indir = "/home/algernon/OutputFiles"
ehists = []
filenames = glob(indir+"/EvoHist*")
with tqdm(total=len(filenames)) as pbar:
    for filename in filenames:
        ehists.append(np.loadtxt(filename))
        pbar.update()

np.savez_compressed("ehists", ehists)
len(ehists)

100%|██████████| 24569/24569 [05:41<00:00, 72.01it/s]
/home/algernon/miniconda3/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


24569

In [14]:
ehists = np.load("EHISTS/ehists_1e6.npz", allow_pickle=True)['arr_0']

In [28]:
other = 0
aic = 0
for ehist in ehists:
    for i in range(len(ehist)):
        if (ehist[i,5]==13 and ehist[i-1,5]!=13 and ehist[i-1,5]!=12) or (ehist[i,8]==13 and ehist[i-1,8]!=13 and ehist[i-1,8]!=12):
            other += 1
        if (ehist[i,5]==13 and ehist[i-1,5]==12) or (ehist[i,8]==13 and ehist[i-1,8]==12):
            aic += 1
aic, other, len(ehists)

(2158, 22411, 24569)

In [29]:
M_bulge = 1.55e10
M_bulge_low = 1.4e10
M_bulge_high = 1.7e10
# M_sim = 7.3e8
M_sim = 1e6
k_bulge = M_bulge/M_sim

In [ ]:
def find_nearest(array, value):
        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin()
        return idx
    
class Distributions:
    def process(self, i, B, a, eta, eta_g):
        G = 1.3218607e+26               # km**3 * MSun**(-1) * yr**(-2)
        G_ = 1.3218607e+26/(695700)**3               # km**3 * MSun**(-1) * yr**(-2)
        g = 6.67408e-11 |(units.m**3 * units.kg**(-1) * units.s**(-2))
        G = g.value_in(units.RSun**3 * units.MSun**(-1) * units.yr**(-2))
        # c = 9.4605284e12                # km/yr
        RSun = 695700   ## km
        c = 2.998e10 ## cm/s
        cc = 2.998e10 |units.cm / units.s  ## cm/s
        
        delKE_acc = 0
        delKE_prop = 0
        delKE_mdb = 0
        
#         i = aic_indices[j]
        
        t = np.arange(0, t_end, dt)
        time_i = ehists[i][:,0]
        self.birthtimes.append(time_i[0])
        types_i = ehists[i][:,5]
        for k in range(len(types_i)):
            if types_i[k] == 13 and types_i[k-1] == 12:
                aic_ind = k
                self.t_aic.append(time_i[aic_ind])
                break

        begining = True 
        j = 0
#         self.pp = []
#         self.ll = []
#         B = ehists[i][-1,14]
#         B = B_list[i]
#         B = random.choice(B_sim_chris)
#         a = a_list[i]
        
        a_d_old = ehists[i][j,2]
#         self.dat[i].append([B, a])
        mic = False
        for t_i in t:
            j_old = j
            j = find_nearest(time_i, t_i/1e6)
            if ehists[i][j,2] == ehists[i][j-1,2]:
                self.mic[int(t_i/dt)]+=1
                mic = True
            if j==j_old:
                mdot = 0
                mloss = 0
            else:
                mdot = ehists[i][j,6]
                mloss = ehists[i][j,10]
            a_d = ehists[i][j,2]
            M1 = ehists[i][j,3]
            M2 = ehists[i][j,7]
            if a_d == a_d_old:
                cc = 9.4605284e12                # km/yr
                adot_grav = - (64/5) * (G**3 * M1*M2*(M1+M2)) / (cc**5 * (a_d_old*RSun)**3)
                a_d = a_d_old + adot_grav/RSun * dt
            if j ==j_old:
                P_orb = None
            else:
                P_orb = 2*np.pi* np.sqrt(a_d**3/(G*(M1+M2)))*525600     ##minutes
            a_d_old = a_d
#             if types_i[j] == 13 and i in aic_indices:
            if types_i[j] == 13:
                w_breakup = np.sqrt(G*ehists[i][j,3]/(ehists[i][j,4]*RSun)**3)
                P_breakup = ( 2 * np.pi /(w_breakup) ) * 3.154e+7    #second
                if accretion == False:
                    mass = ehists[i][j,3] * 1.98e33
                    radius = ehists[i][j,4] * 695700 * 1e5
                    c = 2.998e10 ## cm/s
                    I = (0.4)*(mass*radius**2)      ##g cm^2
                    if begining == True:
#                         P_old = ehists[i][j,12]
#                         P = ehists[i][j,12]
#                         w_ns = ( 2 * np.pi / ( P) ) * 3.154e+7
                        l = 1
                        w_wd = ( 2 * np.pi / (ehists[i][j-l,12]) ) * 3.154e+7  
                        w_ns = ( (ehists[i][j-l,3] / ehists[i][j,3]) * ( ehists[i][j-l,4]/ehists[i][j,4] )**2 ) * w_wd
                        P = ( 2 * np.pi / ( w_ns) ) * 3.154e+7  #seconds
                        if P<0.8e-3:
                            P = ehists[i][j,12]
                            w_ns = ( 2 * np.pi / ( P) ) * 3.154e+7
                            if P<0.8e-3:
                                P = 1e-3
                                w_ns = ( 2 * np.pi / ( P) ) * 3.154e+7
                        self.Pbirth.append(P)
                        alp = np.random.random()
                        mass =  ehists[i][j,3] * 1.989e+33  ##g
                        r =  ehists[i][j,4] * 6.957e+10     ##cm
                        I = (0.4)*(mass*r**2)      ##g cm^2
#                         self.KE0.append(0.5*I*w_ns**2/3.154e+7**2)
                        KE_ = 0.5*I*w_ns**2
                        
                        self.mass.append(ehists[i][j,3])
            
                        self.Pbirth.append(P)
                        alp = np.random.random()
#                         alp = 1e-6
                        P, w, w_s, P_dot, flag, alp, P_dot_mdb1, delKE_acci, delKE_propi, delKE_mdbi = magnetic_braking(P, 
                                                                500e-2/3.154e+7, B, ehists[i][j,3], mdot, ehists[i][j,4], a, True, alp, ehists[i][j-1,12], ehists[i][j-1,3], ehists[i][j-1,4])
                        KE__ = 0.5*I*w**2
                        self.delKE_gw.append((KE__ - KE_)/3.154e+7**2)
                        self.KE0.append(0.5*I*w**2/3.154e+7**2)

                        
                        P, w, w_s, P_dot, flag, alp, P_dot_mdb2, delKE_acci, delKE_propi, delKE_mdbi = magnetic_braking(P, 
                                                                dt, B, ehists[i][j,3], mdot, ehists[i][j,4], a, False, alp)
                        delKE_acc += delKE_acci
                        delKE_prop += delKE_propi
                        delKE_mdb += delKE_mdbi
                        
#                         P_dot_mdb = P_dot_mdb1+P_dot_mdb2
                        P_dot_mdb = P_dot_mdb2
                        begining = False
                        P_old = P 
                        P_dot_old = P_dot
                        begining = False
                    else:
                        P = P_old
                        P, w, w_s, P_dot, flag, alp, P_dot_mdb, delKE_acci, delKE_propi, delKE_mdbi = magnetic_braking(P, 
                                                                                                    dt, B, ehists[i][j,3], mdot, ehists[i][j,4], a, False, alp)
                        delKE_acc += delKE_acci
                        delKE_prop += delKE_propi
                        delKE_mdb += delKE_mdbi
                        P_old = P 
                        P_dot_old = P_dot
                    if P<40e-3 and P_dot > 0 and P_dot_old > 0:
                        self.n[int(t_i/dt)].append( 2 - (P*(P_dot-P_dot_old)/(dt * 3.154e+7))/P_dot**2 )
                    P_dot_old = P_dot

                if accretion == True:
                    if begining == True:
                        l = 1
                        w_wd = ( 2 * np.pi / (ehists[i][j-l,12]) ) * 3.154e+7  
                        w_ns = ( (ehists[i][j-l,3] / ehists[i][j,3]) * ( ehists[i][j-l,4]/ehists[i][j,4] )**2 ) * w_wd
                        P = ( 2 * np.pi / ( w_ns) ) * 3.154e+7  #seconds
                        if P<0.8e-3:
                            P = ehists[i][j,12]
                            w_ns = ( 2 * np.pi / ( P) ) * 3.154e+7
                            if P<0.8e-3:
                                P = np.random.random()*100e-3
                                w_ns = ( 2 * np.pi / ( P) ) * 3.154e+7
                        
#                         P = ehists[i][j,12]
#                         w_ns = ( 2 * np.pi / ( P) ) * 3.154e+7
                        mass =  ehists[i][j,3] * 1.989e+33  ##g
                        r =  ehists[i][j,4] * 6.957e+10     ##cm
                        I = (0.4)*(mass*r**2)      ##g cm^2
#                         self.KE0.append(0.5*I*w_ns**2/3.154e+7**2)
                        KE_ = 0.5*I*w_ns**2
                        
                        self.mass.append(ehists[i][j,3])
            
                        self.Pbirth.append(P)
                        alp = np.random.random()
#                         alp = 1e-6
                        P, w, w_s, P_dot, flag, alp, P_dot_mdb1, delKE_acci, delKE_propi, delKE_mdbi = magnetic_braking(P, 
                                                                500e-2/3.154e+7, B, ehists[i][j,3], mdot, ehists[i][j,4], a, True, alp, ehists[i][j-1,12], ehists[i][j-1,3], ehists[i][j-1,4])
                        KE__ = 0.5*I*w**2
                        self.delKE_gw.append((KE__ - KE_)/3.154e+7**2)
#                         delKE_acc += delKE_acci
#                         delKE_prop += delKE_propi
#                         delKE_mdb += delKE_mdbi
#                         if j==k:
                        self.KE0.append(0.5*I*w**2/3.154e+7**2)
#                         if 0.5*I*w**2/3.154e+7**2 > 1e52:
#                             print(P, mass, r)
                        
                        P, w, w_s, P_dot, flag, alp, P_dot_mdb2, delKE_acci, delKE_propi, delKE_mdbi = magnetic_braking(P, 
                                                                dt, B, ehists[i][j,3], mdot, ehists[i][j,4], a, False, alp)
                        delKE_acc += delKE_acci
                        delKE_prop += delKE_propi
                        delKE_mdb += delKE_mdbi
                        
#                         P_dot_mdb = P_dot_mdb1+P_dot_mdb2
                        P_dot_mdb = P_dot_mdb2
                        begining = False
                        P_old = P 
                        self.aic.append(self.aic[-1]+1)
                    else: 
                        P = P_old
                        P, w, w_s, P_dot, flag, alp, P_dot_mdb, delKE_acci, delKE_propi, delKE_mdbi = magnetic_braking(P, 
                                                                                                    dt, B, ehists[i][j,3], mdot, ehists[i][j,4], a, False, alp)
                        delKE_acc += delKE_acci
                        delKE_prop += delKE_propi
                        delKE_mdb += delKE_mdbi
                        P_old = P 
#                 self.pp.append(P)
                f, alpha, beta = 0.0122, -2.12, 0.82                                   
                L_gamma = 6.8172e35 * f * (P/1e-3)**alpha * (P_dot_mdb/1e-20)**beta 
#                 self.ll.append(L_gamma)
#                 self.dat[i].append([t_i/1e9, P, P_dot_mdb, ehists[i][j,3],
#                                     ehists[i][j,7], ehists[i][j,9], a_d_old])

                        
                        
                        
                        

                
#                 print(delKE_mdb/(dt*3.154e+7)**2)
#                 if P<40e-3 and P_dot > 0:
                if P_dot > 0:
                    mass =  ehists[i][j,3] * 1.989e+33  ##g
                    r =  ehists[i][j,4] * 6.957e+10     ##cm
                    I = (0.4)*(mass*r**2)      ##g cm^2
                    E_dot_mdb = 4 * np.pi**2 * I * P_dot_mdb/P**3     # g cm^2 / s^3 = ergs/s
                    E_dot = 4 * np.pi**2 * I * P_dot/P**3     # g cm^2 / s^3 = ergs/s
#                     ag, bg, dg = 1.2, 0.1, 0.5
#                     aE, bE = 0.18, 2.83
#                     sigma = 0.23
#                     mu = aE * np.log10(E_dot/10**(34.5)) + bE
#                     s = np.random.normal(mu, sigma, int(1e3))
#                     x = np.linspace(min(10**s), max(10**s), int(1e3))
#                     p = (np.sqrt(2*np.pi)*sigma)**-1 * np.exp( -(np.log10(x)-np.log10(10**mu))**2 /(2*sigma**2)  )
#                     p = p/len(p)/np.mean(p)
#                     Ecut = np.random.choice(x,p=p)*0.00160218  ##erg
#                     L_gamma=eta_g * Ecut**ag * B**bg * E_dot**dg
                    f, alpha, beta = 0.0122, -2.12, 0.82                                    ##slot-gap two-pole caustic (TPC) Gonthier 2018
                    L_gamma = 6.8172e35 * f * (P/1e-3)**alpha * (P_dot_mdb/1e-20)**beta 
                    self.dNdL_gamma[int(t_i/dt)].append( L_gamma )
                    self.Power[int(t_i/dt)].append( 1/P**2 )
                    self.P_dist[int(t_i/dt)].append( P )
                    self.P_dot_dist[int(t_i/dt)].append( P_dot )
                    self.P_dot_mdb_dist[int(t_i/dt)].append( P_dot_mdb )
                    self.B_dist[int(t_i/dt)].append( B ) 
#                     if L_gamma>1e33:
#                         self.detectable[int(t_i/dt)].append( True )
#                     else:
#                         self.detectable[int(t_i/dt)].append( False )
                    self.Edot[int(t_i/dt)].append(E_dot)
                    self.Edot_mdb[int(t_i/dt)].append(E_dot_mdb)
                    self.k_donor[int(t_i/dt)].append( ehists[i][j,9] )
#                     self.brake_flags[int(t_i/dt)].append( flag )
#                     T = 2*np.pi * np.sqrt(ehists[i][-1,2]**3/(G_*(ehists[i][j,3]+ehists[i][j,7])))
#                     if T*365<1:
#                         if ehists[i][j,7]>=0.1:
#                             self.redbacks[int(t_i/dt)] += 1
#                         if 0.1>ehists[i][j,7]>=0.01:
#                             self.blackwidows[int(t_i/dt)] += 1
                    if mic == True:
                        self.L_mic[int(t_i/dt)].append( L_gamma )
            
                    

            if ehists[i][j,5] in [10,11,12,13] or ehists[i][j,9] in [10,11,12,13]:  ## if primary is an accreting WD or NS
                if mdot>0:   ##primary accreting
#                     eta = 1.0  ##η_bol ≈ 0.55
                    R_sch = 2 * G * (ehists[i][j,3]) / c**2
                    xi_ = 0.5 * R_sch / (ehists[i][j,4])
                    L = eta * xi_ * abs(mdot) * c**2 
                    L = L * 2e33 * (6.957e+10)**2 / (3.154e+7)**3 
#                     eta = 0.55 * 0.01  ##η_bol ≈ 0.55
#                     L = eta * G * ehists[i][j,3] * abs(ehists[i][j,6]) / (ehists[i][j,4])
#                     L = L * 1.989e33 * (6.957e+10)**2 / (3.154e+7)**3 

                elif mloss>0: ##secondary accreting
#                     eta = 1.0   ##η_bol ≈ 0.55
                    R_sch = 2 * G * (ehists[i][j,7]) / c**2
                    xi_ = 0.5 * R_sch / (ehists[i][j,8])
                    L = eta * xi_ * abs(mloss) * c**2 
                    L = L * 2e33 * (6.957e+10)**2 / (3.154e+7)**3 
#                     eta = 0.55 * 0.01  ##η_bol ≈ 0.55
#                     L = eta * G * ehists[i][j,7] * abs(ehists[i][j,10]) / (ehists[i][j,8])
#                     L = L * 1.989e33 * (6.957e+10)**2 / (3.154e+7)**3 
                else:
                    L = 0
                if L > 0:
                    self.dNdL_x[int(t_i/dt)].append( L ) 
                    if ehists[i][j,5] in [10,11,12]:
                        self.dNdL_xwd[int(t_i/dt)].append( L ) 
                    if ehists[i][j,5] == 13:
                        self.dNdL_xns[int(t_i/dt)].append( L )
                    if L >= 1e36:
                        self.Lx_count[int(t_i/dt)] += 1
                    self.Lx_count_all[int(t_i/dt)] += 1
                    self.P_orb[int(t_i/dt)].append( P_orb )
        
        if types_i[j] == 13:            
            self.delKE_acc.append(delKE_acc/3.154e+7**2)
            self.delKE_prop.append(delKE_prop/3.154e+7**2)
            self.delKE_mdb.append(delKE_mdb/3.154e+7**2)
            mass =  ehists[i][j,3] * 1.989e+33  ##g
            r =  ehists[i][j,4] * 6.957e+10     ##cm
            I = (0.4)*(mass*r**2)      ##g cm^2
            self.KE138.append(0.5*I*w**2/3.154e+7**2)
            if ehists[i][j,7] <0.001:
                self.isolated.append(1)
            else:
                self.isolated.append(0)
                    
                    
    def __init__(self, indices):
        t = np.arange(0, t_end, dt)
        self.dNdL_gamma = [[] for _ in range(len(t))]
        self.Edot = [[] for _ in range(len(t))]
        self.Edot_mdb = [[] for _ in range(len(t))]
        self.L_g_noaic = [[] for _ in range(len(t))]
        self.P_dist = [[] for _ in range(len(t))]
        self.P_dot_dist = [[] for _ in range(len(t))]
        self.P_dot_mdb_dist = [[] for _ in range(len(t))]
        self.Power = [[] for _ in range(len(t))]
        self.B_dist = [[] for _ in range(len(t))]
        self.age = [[] for _ in range(len(t))]
        self.dNdL_x = [[] for _ in range(len(t))]
        self.dNdL_xwd = [[] for _ in range(len(t))]
        self.dNdL_xns = [[] for _ in range(len(t))]
        self.birthtimes = []
        self.detectable = [[] for _ in range(len(t))]
        self.Lx_count = [0 for _ in range(len(t))]
        self.Lx_count_all = [0 for _ in range(len(t))]
        self.t_aic = []
        self.n = [[] for _ in range(len(t))]
        self.Pbirth = []
        self.brake_flags = [[] for _ in range(len(t))]
        self.redbacks = [0 for _ in range(len(t))]
        self.blackwidows = [0 for _ in range(len(t))]
        self.KE0 = []
        self.KE138 = []
        self.delKE_acc = []
        self.delKE_prop = []
        self.delKE_mdb = []
        self.delKE_gw = []
        self.mass = []
        self.finalmass = []
        self.isolated = []
        self.pp = []
        self.ll = []
        self.aic = [0]
        self.mic = [0 for _ in range(len(t))]
        self.dat = [[] for _ in range(len(ehists))]
        self.P_orb = [[] for _ in range(len(t))]
        self.k_donor = [[] for _ in range(len(t))]
        self.L_mic = [[] for _ in range(len(t))]
        
        B_list = np.random.choice(B_sam, len(ehists))
        a_list = np.random.choice(a_sam, len(ehists))
        eta = np.random.choice(etalist, len(ehists))
        eta_g = np.random.choice(eta_sam, len(ehists))
        
        

#         with tqdm(total=len(ehists)) as pbar:
#             for j in range(len(ehists)):
        with tqdm(total=len(indices)) as pbar:
            for j in indices:
                self.process(j, B_list[j], a_list[j], eta[j], eta_g[j])
                pbar.update()
        print("Done!")
#             ncores = 2
#             args = [(ehists, aic_indices[i]) for i in range(len(aic_indices))]
#             print(args)
#             with mp.Pool(processes=ncores) as pool:

#                 for i in enumerate(pool.starmap(self.process, 
#                                             args )):    
#                     pbar.update()

                    

def call_distributions(indices):
    return Distributions(indices)